# Инструменты Data Science: анализ зарплат в IT

## День 5. Результаты анализа вакансий Data Science – чего хотят работодатели?

In [1]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('DataScienceJobs.csv')
df.head()

,name,address,experience,skills,sal_from,sal_to
0,Начинающий специалист по работе с нейросетям,Санкт-Петербург,От 1 года до 3 лет,"Разработка технических заданий, Python, Сбор и...",55000,60000
1,Junior Data Engineer,Нижний Новгород,От 1 года до 3 лет,"SQL, jvm, Hadoop",150000,200000
2,ML-инженер,Москва,Нет опыта,"Знания математики, алгоритмов, структур данных...",150000,170000
3,Data Scientist,Москва,От 1 года до 3 лет,"Python, Machine Learning, Математическое модел...",180000,200000
4,Python Developer (Junior),Томск,От 1 года до 3 лет,"Python, Git, ООП, MySQL",60000,110000


### Вопрос 1. В каких городах больше всего вакансий для Data Scientists?

In [7]:
cities = df['address'].value_counts()[:5]

trace0 = go.Pie(
    name = "",
    values = cities.values,
    labels = cities.index,
    hovertemplate = "<b>%{label}</b><br>Доля: %{percent} </br>Количество: %{value} вакансий"
)
layout= go.Layout(
    title= 'Распределение вакансий по городам'
)
fig = go.Figure(data = [trace0], layout = layout)
fig.show()

### Вопрос 2. Какие навыки требуются на рынке? Чему учиться?

In [17]:
skills = df['skills'].str.cat(sep = ' ').lower().replace(',', '').replace(';', '').split()
skills = [skill for skill in skills if len(skill) > 1]

skills_vis = Counter(skills).most_common(15)
sk = pd.DataFrame(skills_vis, columns = ['skill', 'num'])
sk['skill'] = sk['skill'].str.upper() # приведем все названия к верхнему регистру
sk

,skill,num
0,PYTHON,255
1,SQL,165
2,ДАННЫХ,90
3,DATA,87
4,ML,82
5,LEARNING,80
6,ОПЫТ,76
7,MACHINE,64
8,PYTORCH,63
9,РАБОТЫ,54


In [18]:
trace0 = go.Pie(
    name = "",
    values = sk['num'],
    labels = sk['skill'],
    hovertemplate = "<b>%{label}</b>\n <br>Доля: %{percent} </br>Количество: %{value} упоминаний"
)
layout= go.Layout(
    title= 'Топ-15 ключевых слов в описаниях вакансий'
)
fig = go.Figure(data = [trace0], layout = layout)
fig.show()

### Вопрос 3. Какие зарплаты на рынке? С каким опытом берут на работу?

In [19]:
df.describe().map("{0:,.2f}".format)

,sal_from,sal_to
count,360.00,360.00
mean,"182,694.44","231,416.67"
std,"70,581.28","79,938.65"
min,"45,000.00","55,000.00"
25%,"120,000.00","170,000.00"
50%,"180,000.00","225,000.00"
75%,"242,500.00","290,000.00"
max,"350,000.00","570,000.00"


In [20]:
# разобьем длинные строки навыков на строки покороче, чтобы они лучше отрисовывались

import textwrap
df.skills = df.skills.apply(lambda txt: '<br>'.join(textwrap.wrap(txt, width=50)))

In [21]:
df['skills'][0]

'Разработка технических заданий, Python, Сбор и<br>анализ информации, Midjorney, Внедрение<br>нейросетей, Нейросеть, ChatGPT, Работа в команде,<br>Обучение и развитие, Stabile Diffusion, Работа с<br>большим объемом информации'

In [22]:
fig = px.scatter(data_frame=df, 
                 x='sal_from', 
                 y='sal_to', 
                 color='experience',
                 title='Распределение заработных плат в зависимости от опыта',
                 hover_name='name', 
                 hover_data=['sal_to', 'sal_from', 'skills'], 
                 color_discrete_sequence=px.colors.qualitative.Plotly,
                 category_orders={"experience":
                                  ["Нет опыта", "От 1 года до 3 лет", "От 3 до 6 лет", "Более 6 лет"]},
               labels={'experience': 'Опыт', 
                       'sal_to': 'Верхняя вилка зарплаты', 
                       'sal_from': 'Нижняя вилка зарплаты',
                       'skills': 'Требуемые навыки'}
              ).update_traces(marker_size=10)
fig.show()